# Restoration


## Mounting Google Drive to access Colaboratory

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive/DL-RP41-Deep-Image-Prior/

## Importing Python Libraries

In [0]:
import matplotlib.pyplot as plot
%matplotlib inline
import numpy as np
import torch as torch
import torch.optim as optim
from PIL import Image as Image
from PIL import ImageDraw as imDraw
from PIL import ImageFont as imFont
import os
import random
import torchvision

from util.common_utils import *
from models.skip_architecture import SkipArchitecture

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True

# Steps

## 1. Image Selection

In [0]:
base_path = 'data/restoration/goi.png'
image = Image.open(fp=base_path)


## 2. Setting Up Image and Mask

In [0]:
#converting the PIL Image into an ndarray
image_np = image_to_ndarray(image)

image_mask = get_mask(image,2,0.55)
image_mask_np = image_to_ndarray(image_mask)

corrupted = image_np * image_mask_np

### 2.1 Displaying Images

In [0]:
plot.imshow(image_np.transpose(1,2,0))

In [0]:
plot.imshow(image_mask_np.transpose(1,2,0))

In [0]:
plot.imshow(corrupted.transpose(1,2,0))

## 3. Setting up Network Params

In [0]:
# Network params tuned according to supmat of paper
params = (0.1) * torch.rand((1,32,512,512))

num_iter = 2000
LR = 0.01
reg_noise_std = 0.00
input_depth = 32

net = SkipArchitecture(input_channels=input_depth, output_channels=image_np.shape[0], 
               filters_down = [128] * 5,
               filters_up =   [128] * 5,
               filters_skip =    [128] * 5,  
               kernel_size_up = [3] * 5, kernel_size_down = [3] * 5, 
               upsample_mode='nearest', kernel_size_skip=[1] * 5)


net = net.type(torch.cuda.FloatTensor)

In [0]:
# Parameters

s  = sum(np.prod(list(p.size())) for p in net.parameters())
print('Number of parameters: ',s)

In [0]:
# Define Loss function

error = torch.nn.MSELoss().type(torch.cuda.FloatTensor)

In [0]:
# Defining the image variables as pyTorch Tensors
image_t = torch.from_numpy(image_np)
image_mask_t = torch.from_numpy(image_mask_np)

## 4. Executing

### 4.1 Checking GPU Availability

In [0]:
 gpu = torch.cuda.is_available()
 gpu_on = torch.cuda.is_initialized()
 print(gpu,gpu_on)
 if gpu == True:
   net = net.cuda()
   params = params.cuda()
   image_t = image_t.cuda()
   image_mask_t = image_mask_t.cuda()

### 4.2 Main Loop

In [0]:
optimizer = torch.optim.Adam(net.parameters(),lr=LR)

for i in range(num_iter):
  optimizer.zero_grad()

  out = net.forward(params)
  total_loss = error(out * image_mask_t, image_t * image_mask_t)
  total_loss.backward()

  psrn_masked = compare_psnr(corrupted, out.detach().cpu().numpy()[0] * image_mask_np) 
  psrn = compare_psnr(image_np, out.detach().cpu().numpy()[0]) 




  optimizer.step()
  if i % 200 == 0:
     print (i, total_loss.item(), psrn_masked, psrn)


  params = params + (1/(30))*torch.randn_like(params)



## 5. Display Result

In [0]:
image_tensor = net.forward(params)
result_np = image_tensor.detach().cpu().numpy()[0]
psnr = compare_psnr(corrupted,result_np * image_mask_np)
print('psnr = ',psnr)
plot.imshow(result_np.transpose(1,2,0))